In [ ]:
"""下载
LLaMA-Factory 用于微调
DISC-Law-SFT 法律数据
"""
!git clone https://github.com/hiyouga/LLaMA-Factory
!git clone https://huggingface.co/datasets/ShengbinYue/DISC-Law-SFT

In [ ]:
"""在LLaMA-Factory中添加DISC-Law-SFT 法律数据
"""
!apt-get install -y jq
!cp /content/DISC-Law-SFT/DISC-Law-SFT-Pair.jsonl /content/LLaMA-Factory/data/
!cp /content/DISC-Law-SFT/DISC-Law-SFT-Triplet-released.jsonl /content/LLaMA-Factory/data/
!jq '.law_sft_pair={"file_name": "DISC-Law-SFT-Pair.jsonl", "columns": {"prompt": "input", "response": "output"}}' /content/LLaMA-Factory/data/dataset_info.json > new_dataset_info.json
!cp  new_dataset_info.json /content/LLaMA-Factory/data/dataset_info.json
!jq '.law_sft_triplet={"file_name": "DISC-Law-SFT-Triplet-released.jsonl", "columns": {"prompt": "input", "response": "output"}}' /content/LLaMA-Factory/data/dataset_info.json > new_dataset_info.json
!cp  new_dataset_info.json /content/LLaMA-Factory/data/dataset_info.json

In [ ]:
"""安装依赖
"""
%cd /content/LLaMA-Factory
!pip install -r requirements.txt
!pip install einops
!pip install transformers==4.34.0
!pip install deepspeed

In [ ]:
"""替换自我认知self_cognition数据集中的名称
"""
%cd /content/LLaMA-Factory
!sed -i 's/<NAME>/法律AI/g' data/self_cognition.json
!sed -i 's/<AUTHOR>/billvsme/g' data/self_cognition.json

In [ ]:
"""生成deepspeed配置文件
"""
%cd /content/LLaMA-Factory
!echo '''{\
  "train_batch_size": "auto",\
  "train_micro_batch_size_per_gpu": "auto",\
  "gradient_accumulation_steps": "auto",\
  "gradient_clipping": "auto",\
  "zero_allow_untested_optimizer": true,\
  "fp16": {\
    "enabled": "auto",\
    "loss_scale": 0,\
    "initial_scale_power": 16,\
    "loss_scale_window": 1000,\
    "hysteresis": 2,\
    "min_loss_scale": 1\
  },\
  "zero_optimization": {\
    "stage": 2,\
    "offload_optimizer": {\
      "device": "cpu",\
      "pin_memory": true\
    },\
    "allgather_partitions": true,\
    "allgather_bucket_size": 5e8,\
    "reduce_scatter": true,\
    "reduce_bucket_size": 5e8,\
    "overlap_comm": false,\
    "contiguous_gradients": true\
  }\
}''' > ds_config.json

In [ ]:
"""训练
指令监督微调，lora方式，使用self_cognition和law_sft_triplet前1000数据集，大概需要60分钟

由于没有对话历史，template使用vanilla

使用deepspeed stage2，offload_optimizer -> cpu节省显存，cutoff_len 可以到1664，在多就要爆显存了
"""
%cd /content/LLaMA-Factory
!rm -rf saves/Phi1.5-1.3B/lora/law
!deepspeed --num_gpus 1 --master_port=9901 src/train_bash.py \
    --deepspeed ds_config.json \
    --stage sft \
    --model_name_or_path microsoft/phi-1_5 \
    --do_train True \
    --finetuning_type lora \
    --template vanilla \
    --flash_attn False \
    --shift_attn False \
    --dataset_dir data \
    --dataset self_cognition,law_sft_triplet \
    --cutoff_len 1664 \
    --learning_rate 2e-04 \
    --num_train_epochs 5.0 \
    --max_samples 1000 \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --lr_scheduler_type cosine \
    --max_grad_norm 1.0 \
    --logging_steps 5 \
    --save_steps 1000 \
    --warmup_steps 0 \
    --neft_alpha 0 \
    --train_on_prompt False \
    --upcast_layernorm False \
    --lora_rank 8 \
    --lora_dropout 0.1 \
    --lora_target Wqkv \
    --resume_lora_training True \
    --output_dir saves/Phi1.5-1.3B/lora/law \
    --fp16 True \
    --plot_loss True

/content/LLaMA-Factory
[2023-11-07 03:32:23,110] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2023-11-07 03:32:26,052] [WARNING] [runner.py:203:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
[2023-11-07 03:32:26,052] [INFO] [runner.py:570:main] cmd = /usr/bin/python3 -u -m deepspeed.launcher.launch --world_info=eyJsb2NhbGhvc3QiOiBbMF19 --master_addr=127.0.0.1 --master_port=9901 --enable_each_rank_log=None src/train_bash.py --deepspeed ds_config.json --stage sft --model_name_or_path microsoft/phi-1_5 --do_train True --finetuning_type lora --template vanilla --flash_attn False --shift_attn False --dataset_dir data --dataset self_cognition,law_sft_triplet --cutoff_len 1664 --learning_rate 2e-04 --num_train_epochs 5.0 --max_samples 1000 --per_device_train_batch_size 1 --per_device_eval_batch_size 1 --gradient_accumulation_steps 1 --lr_scheduler_type cosine --max_grad_norm 1.0 --logging_steps 5

In [ ]:
"""导出模型
"""
%cd /content/LLaMA-Factory
!mkdir out_model
!python src/export_model.py \
    --model_name_or_path  microsoft/phi-1_5 \
    --template default \
    --finetuning_type lora \
    --checkpoint_dir saves/Phi1.5-1.3B/lora/law \
    --export_dir out_model/law

In [ ]:
"""查看模型效果
"""

%cd /content/LLaMA-Factory

import os
from threading import Thread

import torch
from transformers.generation.streamers import TextIteratorStreamer
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

torch.set_default_device("cuda")

model_name = "/content/LLaMA-Factory/out_model/law/"
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
model.generation_config = GenerationConfig.from_pretrained(model_name)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token


def stream(prompt):
    inputs = tokenizer(prompt, return_tensors='pt', padding=True, return_attention_mask=True)
    streamer = TextIteratorStreamer(
        tokenizer,
        decode_kwargs={"skip_special_tokens": True})
    Thread(
        target=model.generate, kwargs=dict(
            inputs, streamer=streamer,
            max_new_tokens=256)
    ).start()

    first = True
    for text in streamer:
        if first and text:
            first = False
            continue

        if not text:
            continue
        if tokenizer.eos_token in text:
            break

        yield text


def main():
    welcome_prompt = "欢迎使用 法律AI 模型，输入内容即可进行对话，clear 清空对话历史，stop 终止程序"
    print(welcome_prompt)
    while True:
        query = input("\n用户：")
        if query.strip() == "stop":
            break
        if query.strip() == "clear":
            os.system("clr")
            print(welcome_prompt)
            continue
        print("\n法律AI：", end="")
        for text in stream(query):
            print(text, end="", flush=True)
        print("")



main()

In [ ]:
"""保存训练好的模型到Google云盘
"""
%cd /content
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
!mkdir /content/drive/MyDrive/llm_model/
!cp -r /content/LLaMA-Factory/out_model/law /content/drive/MyDrive/llm_model/